In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import csv
import time
import random
from collections import defaultdict

columns = defaultdict(list)

with open('C:/Users/KNguyen/NCUC_master_list.csv') as ncuc:
    reader = csv.DictReader(ncuc) # read rows into a dictionary format
    for row in reader: # read a row as {column1: value1, column2: value2,...}
        for (k,v) in row.items(): # go over each column name and value 
            columns[k].append(v) # append the value into the appropriate list
                                 # based on column name k

In [4]:
docket_details = columns['Detail']
format_details = [x for x in docket_details if x!='none']
print (len(docket_details))
print (len(format_details))

2344
2305


In [5]:
driver = webdriver.Chrome("C:/Users/KNguyen/chromedriver.exe")

with open('C:/Users/KNguyen/NCUC_Details.csv', 'w', newline='') as ncuc:
    a = csv.writer(ncuc, delimiter=',')
    bill_rows = [['ID', 'Date opened', 'Status', 'Description','Sypnosis','Companies Involved']]
    for docket in format_details:
        try:
            driver.get(docket)
            driver.implicitly_wait(1)

            results_dirty = driver.find_element_by_css_selector("#ctl00_ContentPlaceHolder1_PortalPageControl1_ctl38_docketDetailsDiv")
            results_html = results_dirty.get_attribute("innerHTML")
            results_segs = results_html.split("<tr>")

            lion = []
            for bit in results_segs:
                format_bit = bit.replace('</span>','">').split('">')
                lion.append(format_bit)

            docket_id = lion[1][5]

            date_opened = lion[2][5]

            docket_status = lion[3][5]

            docket_desp = lion[4][6]

            docket_syp = lion[5][5]

            final_companies = []
            companies_involved = ''
            for bit in lion[6]:
                format_bit = bit.split('</p>')
                final_companies.append(format_bit)
            for x in range (5,len(final_companies)):
                companies_involved +=(final_companies[x][0]) + ', '

            newrow = [docket_id, date_opened, docket_status, docket_desp, docket_syp, companies_involved]
            bill_rows.append(newrow)
        
        except NoSuchElementException:
            driver.refresh()
            continue
        #print()
        #print()
    a.writerows(bill_rows)